In [3]:
import pandas as pd
import numpy as np
import requests
import json
import os
from config import geoapify_key
import zipfile


### Data Processing (collection and manipulation of data)

#### Collection

In [ ]:
# Zip File Extraction
    # ? Insead of extracting one zip file at a given time, we will extract all the zip files using a for loop
def unzip_files(zip_path, extract_path):
    for zip_path in zip_paths:
        with zipfile.ZipFile (zip_path, 'r') as zObject:
        # * Extracting all memebters of the zip files into a specific location
            zObject.extractall(
                path = extract_path
            )
# * Usage zip function - Variables:
zip_paths = ["/Users/galbeeir/Desktop/git/crime_analysis/Data source zips/Datasets_1.zip", "/Users/galbeeir/Desktop/git/crime_analysis/Data source zips/Datasets_2.zip"]
extract_path = "/Users/galbeeir/Desktop/git/crime_analysis/"

# * Executing the zip function
unzip_files(zip_paths, extract_path)


In [ ]:
# * List of directories
datasets_1 = "/Users/galbeeir/Desktop/git/crime_analysis/Datasets_1/"
datasets_2 = "/Users/galbeeir/Desktop/git/crime_analysis/Datasets_2/"

# * List all the files from the directory
file_list_1 = os.listdir(datasets_1)
file_list_2 = os.listdir(datasets_2)

# * Merge all files stored in file_list
crime_df = pd.DataFrame()

# * Merge all files stored in file_list_1
for file in file_list_1:
    if file.endswith(".csv"):
        file_path = os.path.join(datasets_1, file)
        df = pd.read_csv(file_path, delimiter=',', encoding='utf-8')
        crime_df = pd.concat([crime_df, df])

# * Merge all files stored in file_list_2
for file in file_list_2:
    if file.endswith(".csv"):
        file_path = os.path.join(datasets_2, file)
        df = pd.read_csv(file_path, delimiter=',', encoding='utf-8')
        crime_df = pd.concat([crime_df, df])
    
     
crime_df.head()

#### Key definitions
1. Ward = City council (There are **50 in Chicago**) 
2. Community Areas = There are **77 commnuity areas** in Chicago 
    * => [Chicago - Community Areas & Wards](https://www.chicago.gov/city/en/depts/dgs/supp_info/citywide_maps.html)
3. District = There are **25 police districts** in Chicago => [Police Districts](https://home.chicagopolice.org/about/police-districts/)

**Data processing actions:**
Convert the community areas numbers to names=>
- Method = using pd.read_html - using the following link => [Wikipadia - community areas in Chicago](https://en.wikipedia.org/wiki/Community_areas_in_Chicago)

*Note:*<br>
To find out more about *Ward, Community Areas, and Police Distrits*, please click here ->[`resources`](https://github.com/Kokolipa/crime_analysis/tree/main/Resources)
<br>

In [ ]:
# Importing Chicago community area names and codes from wikipedia
chi_community_areas = (pd.read_html("https://en.wikipedia.org/wiki/Community_areas_in_Chicago")[0]
 .droplevel(level=0, axis=1)
 .set_index('No.')
 .drop(['(sq mi.)', '(/sq mi.)', '(km2)', '(/km2)'], axis=1)
 .rename(columns={'.mw-parser-output .nobold{font-weight:normal}(2020)[10]': 'Population'}))

chi_community_areas.head()

In [ ]:
# Removing the Total row an the button of the dataframe
chi_community_areas = chi_community_areas.iloc[:-1]

In [ ]:
# Changing the dtype of 
chi_community_areas.index = chi_community_areas.index.astype('Int8')
chi_community_areas.head()

In [ ]:
# Transforming the dtype of Community area from crime_df
crime_df['Community Area'] = crime_df['Community Area'].astype('Int8')

In [ ]:
# Validation transformation
crime_df['Community Area'].dtype

In [ ]:
# Merging both of the dataframes
crime_df = pd.merge(
    left = crime_df,
    right = chi_community_areas,
    how = 'left',
    left_on= ['Community Area'],
    right_on= ['No.'],
)

In [ ]:
crime_df.columns

In [ ]:
# Renaming columns
crime_df.drop(['Unnamed: 0', 'Community Area', 'Population', 'Description', 'IUCR', 'Beat'], axis=1, inplace=True)

In [ ]:
# Reorginising the columns of the dataset
crime_df = crime_df.reindex(labels=['ID', 'Case Number', 'Date', 'Year','Updated On','Block', 'Primary Type', 'Location Description', 'Arrest', 'Domestic', 'District', 'Ward', 'Name', 'Latitude', 'Longitude'], axis=1)

In [ ]:
# Dropping irrelevant columns
crime_df.rename(columns={'Name': 'Community Area', 'District': 'Police District'}, inplace=True)

#### Manipulation

In [ ]:
# Search key method (ABC)
crime_df.loc[crime_df['Primary Type'].str.startswith('C')]['Primary Type'].value_counts()

In [ ]:
crime_df['Primary Type'].replace({'CRIM SEXUAL ASSAULT': 'CRIMINAL SEXUAL ASSAULT'}, inplace=True)

In [ ]:
# Replacing NaN values in the Location Description column with 'BLANK' value
crime_df['Location Description'].fillna('Blank', inplace=True)

In [ ]:
# Search key method (ABC)
crime_df.loc[crime_df['Location Description'].str.startswith('Z')]['Location Description'].value_counts()

In [ ]:
# Replacing all duplicated / look alike values
crime_df['Location Description'].replace({'AIRPORT TERMINAL UPPER LEVEL - SECURE AREA': 'AIRPORT',
                                          'AIRPORT TERMINAL LOWER LEVEL - NON-SECURE AREA': 'AIRPORT',
                                          'AIRPORT PARKING LOT': 'AIRPORT',
                                          'AIRPORT BUILDING NON-TERMINAL - NON-SECURE AREA': 'AIRPORT',
                                          'AIRPORT TERMINAL UPPER LEVEL - NON-SECURE AREA': 'AIRPORT',
                                          'AIRPORT EXTERIOR - NON-SECURE AREA': 'AIRPORT',
                                          'AIRPORT BUILDING NON-TERMINAL - SECURE AREA': 'AIRPORT',
                                          'AIRPORT EXTERIOR - SECURE AREA': 'AIRPORT',
                                          'AIRPORT/AIRCRAFT': 'AIRPORT',
                                          'AIRPORT VENDING ESTABLISHMENT': 'AIRPORT',
                                          'AIRPORT TRANSPORTATION SYSTEM (ATS)': 'AIRPORT',
                                          'AIRPORT TERMINAL MEZZANINE - NON-SECURE AREA ': 'AIRPORT',
                                          'BOAT / WATERCRAFT': 'BOAT/WATERCRAFT',
                                          'CTA TRAIN': 'CRIMINAL TRIBES ACT',
                                          'CTA BUS': 'CRIMINAL TRIBES ACT',
                                          'CTA PLATFORM': 'CRIMINAL TRIBES ACT',
                                          'CTA STATION': 'CRIMINAL TRIBES ACT',
                                          'CTA BUS STOP': 'CRIMINAL TRIBES ACT',
                                          'CTA PARKING LOT / GARAGE / OTHER PROPERTY': 'CRIMINAL TRIBES ACT',
                                          'CTA GARAGE / OTHER PROPERTY': 'CRIMINAL TRIBES ACT',
                                          'CTA TRACKS - RIGHT OF WAY': 'CRIMINAL TRIBES ACT',
                                          'CTA "L" TRAIN': 'CRIMINAL TRIBES ACT',
                                          'CTA "L" PLATFORM': 'CRIMINAL TRIBES ACT',
                                          'CTA PROPERTY': 'CRIMINAL TRIBES ACT',
                                          'CTA SUBWAY STATION': 'CRIMINAL TRIBES ACT',
                                          'CHA APARTMENT': 'CHICAGO HOUSING AUTHORITY',
                                          'CHA PARKING LOT / GROUNDS': 'CHICAGO HOUSING AUTHORITY',
                                          'CHA HALLWAY / STAIRWELL / ELEVATOR': 'CHICAGO HOUSING AUTHORITY',
                                          'CHA HALLWAY/STAIRWELL/ELEVATOR': 'CHICAGO HOUSING AUTHORITY',
                                          'CHA PARKING LOT': 'CHICAGO HOUSING AUTHORITY',
                                          'CHA GROUNDS': 'CHICAGO HOUSING AUTHORITY',
                                          'CHA LOBBY': 'CHICAGO HOUSING AUTHORITY',
                                          'CHA PLAY LOT': 'CHICAGO HOUSING AUTHORITY',
                                          'CHA HALLWAY': 'CHICAGO HOUSING AUTHORITY',
                                          'CHA ELEVATOR': 'CHICAGO HOUSING AUTHORITY',
                                          'CHURCH/SYNAGOGUE/PLACE OF WORSHIP': 'CHURCH',
                                          'CHURCH / SYNAGOGUE / PLACE OF WORSHIP': 'CHURCH',
                                          'COLLEGE / UNIVERSITY - GROUNDS': 'COLLEGE/UNIVERSITY',
                                          'COLLEGE/UNIVERSITY GROUNDS': 'COLLEGE/UNIVERSITY',
                                          'FACTORY / MANUFACTURING BUILDING':'FACTORY/MANUFACTURING BUILDING',
                                          'GAS STATION DRIVE/PROP.': 'GAS STATION',
                                          'GOVERNMENT BUILDING / PROPERTY':'GOVERNMENT BUILDING/PROPERTY',
                                          'HOTEL / MOTEL': 'HOTEL/MOTEL',
                                          'HOSPITAL BUILDING / GROUNDS': 'HOSPITAL',
                                          'HOSPITAL BUILDING/GROUNDS': 'HOSPITAL',
                                          'LAKEFRONT / WATERFRONT / RIVERBANK':'LAKEFRONT/WATERFRONT/RIVERBANK',
                                          'MOVIE HOUSE / THEATER': 'MOVIE HOUSE/THEATER',
                                          'MEDICAL / DENTAL OFFICE':'MEDICAL/DENTAL OFFICE',
                                          'NURSING / RETIREMENT HOME':'NURSING HOME/RETIREMENT HOME',
                                          'NURSING HOME':'NURSING HOME/RETIREMENT HOME',
                                          'OTHER (SPECIFY)':'OTHER',
                                          'OTHER RAILROAD PROPERTY / TRAIN DEPOT':'OTHER RAILROAD PROP/TRAIN DEPOT',
                                          'OTHER RAILROAD PROP / TRAIN DEPOT':'OTHER RAILROAD PROP/TRAIN DEPOT',
                                          'PARKING LOT / GARAGE (NON RESIDENTIAL)': 'PARKING LOT/GARAGE',
                                          'PARKING LOT/GARAGE(NON.RESID.)': 'PARKING LOT/GARAGE',
                                          'POLICE FACILITY / VEHICLE PARKING LOT': 'POLICE FACILITY',
                                          'POLICE FACILITY/VEH PARKING LOT':'POLICE FACILITY',
                                          'RESIDENCE - PORCH / HALLWAY': 'RESIDENCE',
                                          'RESIDENCE - YARD (FRONT / BACK)': 'RESIDENCE',
                                          'RESIDENCE - GARAGE': 'RESIDENCE',
                                          'RESIDENCE PORCH/HALLWAY': 'RESIDENCE',
                                          'RESIDENTIAL YARD (FRONT/BACK)': 'RESIDENCE',
                                          'RESIDENCE-GARAGE':'RESIDENCE', 
                                          'SCHOOL - PUBLIC BUILDING': 'SCHOOL',
                                          'SCHOOL, PUBLIC, BUILDING': 'SCHOOL',
                                          'SCHOOL - PUBLIC GROUNDS': 'SCHOOL',
                                          'SCHOOL, PUBLIC, GROUNDS': 'SCHOOL',
                                          'SCHOOL - PRIVATE GROUNDS': 'SCHOOL',
                                          'SCHOOL, PRIVATE, BUILDING': 'SCHOOL',
                                          'SCHOOL - PRIVATE BUILDING': 'SCHOOL',
                                          'SCHOOL, PRIVATE, GROUNDS': 'SCHOOL',
                                          'SCHOOL YARD': 'SCHOOL',
                                          'SPORTS ARENA / STADIUM': 'SPORTS ARENA/STADIUM',
                                          'TAVERN / LIQUOR STORE': 'TAVERN/LIQUOR STORE',
                                          'VEHICLE - COMMERCIAL':'VEHICLE-COMMERCIAL',
                                          'VEHICLE - COMMERCIAL: ENTERTAINMENT / PARTY BUS':'VEHICLE-COMMERCIAL',
                                          'VEHICLE - COMMERCIAL: TROLLEY BUS':'VEHICLE-COMMERCIAL',
                                          'VEHICLE-COMMERCIAL - TROLLEY BUS':'VEHICLE-COMMERCIAL',
                                          'VEHICLE - COMMERCIAL':'VEHICLE-COMMERCIAL',
                                          'VEHICLE-COMMERCIAL - ENTERTAINMENT/PARTY BUS':'VEHICLE-COMMERCIAL',
                                          'VEHICLE - OTHER RIDE SHARE SERVICE (LYFT, UBER, ETC.)':'VEHICLE-OTHER RIDE SHARE SERVICE',
                                          'VEHICLE - OTHER RIDE SHARE SERVICE (E.G., UBER, LYFT)':'VEHICLE-OTHER RIDE SHARE SERVICE',
                                          }, inplace=True)

In [ ]:
crime_df.head()

### Memory Optimization

In [ ]:
crime_df.info(memory_usage='deep')

In [ ]:
crime_df.head()

In [ ]:
# Bits - reduction
crime_df = crime_df.astype({'Year': 'Int16',
                'Police District': 'Int8',
                'Ward': 'category',
                'Location Description': 'category',
                'Primary Type': 'category',
                'ID': 'Int32',
                'Latitude': 'float32',
                'Longitude': 'float32'})


In [ ]:
crime_df['Date'] = pd.to_datetime(
    crime_df['Date'],
    errors = 'coerce',
    infer_datetime_format=True, 
)

In [ ]:
crime_df['Updated On'] = pd.to_datetime(
    crime_df['Updated On'],
    errors = 'coerce',
    infer_datetime_format=True, 
    format ='%T'
)

In [ ]:
# Memory consumption reduction = 200 + MB
crime_df.info(memory_usage='deep')

### Data Cleaning

In [ ]:
crime_df.isna().sum()

In [ ]:
crime_df['Ward'].fillna('Blank', inplace=True)

In [ ]:
crime_df['Community Area'].fillna('Blank', inplace=True)

In [ ]:
crime_df.dropna(subset=['Latitude', 'Longitude'], how='any', inplace=True)

#### NOTE:
Because our crime dataset include 17,800 records of NaN values in both Latitude and Longitude we removed them as the purpose of this analysis is to explore the distribution of crime before and after COVID-19

In [ ]:
crime_df.isna().sum()

In [ ]:
crime_df.info()

#### Exporting the data to ZipFile for Analysis

In [ ]:
crime_df.to_csv("/Users/galbeeir/Desktop/git/crime_analysis/crime_cleaned.csv")

In [4]:

def export_to_zip(csv_path, zip_path_1, zip_path_2, compression_level=9):
    # Read the csv file
    crime_df = pd.read_csv(csv_path)

    # Filter the DataFrame based on the conditions
    filtered_1 = crime_df.loc[(crime_df['Year'] == 2019)]
    filtered_2 = crime_df.loc[(crime_df['Year'] == 2020)]
    filtered_3 = crime_df.loc[(crime_df['Year'] == 2021)]
    filtered_4 = crime_df.loc[(crime_df['Year'] == 2022)]
    filtered_5 = crime_df.loc[(crime_df['Year'] == 2023)]

    # Create a ZipFile and add the filtered data as separate CSV files with compression
    with zipfile.ZipFile(zip_path_1, 'w', zipfile.ZIP_DEFLATED, compresslevel=compression_level) as zip_file_1:
        zip_file_1.writestr('filtered_data_1.csv', filtered_1.to_csv(index=False))
        zip_file_1.writestr('filtered_data_2.csv', filtered_2.to_csv(index=False))
    
    with zipfile.ZipFile(zip_path_2, 'w', zipfile.ZIP_DEFLATED, compresslevel=compression_level) as zip_file_2:
        zip_file_2.writestr('filtered_data_3.csv', filtered_3.to_csv(index=False))
        zip_file_2.writestr('filtered_data_4.csv', filtered_4.to_csv(index=False))
        zip_file_2.writestr('filtered_data_5.csv', filtered_5.to_csv(index=False))

csv_cleaned_path = "/Users/galbeeir/Desktop/git/crime_analysis/crime_cleaned.csv"
zip_cleaned_path_1 = "/Users/galbeeir/Desktop/git/crime_analysis/filtered_1.zip"
zip_cleaned_path_2 = "/Users/galbeeir/Desktop/git/crime_analysis/filtered_2.zip"

export_to_zip(csv_cleaned_path, zip_cleaned_path_1, zip_cleaned_path_2)


/var/folders/06/47q5ms9540g4mnz8nnnyl32m0000gn/T/ipykernel_28000/27589574.py:3: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  crime_df = pd.read_csv(csv_path)
